# 🚀 Advanced Agents & MLOps implementations

This notebook provides practical code implementations for the concepts discussed in the `150_ADV_AGENTS_MLOPS_EVALS.md` guide. We focus on implementable patterns while omitting purely architectural or infrastructure-based questions.

## 🛠 Setup

In [ ]:
import os
from typing import List, Dict, Any
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("Setup Complete.")

## 🏗️ Section 1: Advanced RAG Patterns

### Q2: HyDE (Hypothetical Document Embeddings)

In [ ]:
def generate_hyde_document(query: str):
    prompt = f"Write a technical document that answers this query: {query}"
    hypothetical_doc = llm.invoke(prompt).content
    print(f"--- HYPOTHETICAL DOCUMENT ---\n{hypothetical_doc[:150]}...")
    return hypothetical_doc

hyde_doc = generate_hyde_document("How to optimize LLM inference?")

### Q7: Corrective RAG (CRAG)

In [ ]:
def corrective_rag(query: str, retrieved_docs: List[str]):
    evaluation_prompt = f"Are these documents relevant to the query: '{query}'? Docs: {retrieved_docs}. Answer YES or NO."
    relevancy = llm.invoke(evaluation_prompt).content.strip().upper()
    
    if 'YES' in relevancy:
        print("✅ Relevancy high. Using documents.")
        return "AI Answer based on docs."
    else:
        print("❌ Relevancy low. Triggering fallback search.")
        return "Fallback search result."

print(corrective_rag("Apple M3", ["Cooking apples recipe"]))

### Q8: Self-RAG (Self-Grading)

In [ ]:
def self_rag_agent(query: str):
    prompt = f"Answer the query. Append [SATISFACTORY] if sure, or [UNSURE] if you need more info. Query: {query}"
    return llm.invoke(prompt).content

print(self_rag_agent("What is the capital of Mars?"))

## 📈 Section 2: Agent Evaluation (Evals)

### Q33: LLM-as-a-Judge

In [ ]:
def llm_judge(question: str, answer: str, ground_truth: str):
    prompt = f"Question: {question}\nAnswer: {answer}\nTruth: {ground_truth}\nScore 1-10 and justify."
    return llm.invoke(prompt).content

print(llm_judge("What is 2+2?", "It is 5", "4"))

## 🚀 Section 3: MLOps & Versioning

### Q70: Prompt Versioning

In [ ]:
registry = {
    "v1.0": "You are a helpful assistant.",
    "v1.1": "You are a technical expert in Python."
}
def run_versioned(version: str, task: str):
    return llm.invoke([SystemMessage(content=registry[version]), HumanMessage(content=task)]).content

print(run_versioned("v1.1", "Explain decorators.")[:50])

## 🛠️ Section 4: Advanced Prompt Engineering

### Q100: Chain of Verification (CoVe)

In [ ]:
def cove_demo(query: str):
    initial = llm.invoke(query).content
    verification = llm.invoke(f"Critique this for errors: {initial}").content
    final = llm.invoke(f"Fix based on critique: {verification}. Original: {initial}").content
    return final

print(cove_demo("When was the first iPhone released?"))

## 🛡️ Section 5: Security & Cost

### Q127: Cost-per-Task (CPT)

In [ ]:
def estimate_cost(input_tokens: int, output_tokens: int):
    # GPT-4o-mini approx prices
    return (input_tokens * 0.15 / 1e6) + (output_tokens * 0.60 / 1e6)

print(f"Task Cost: ${estimate_cost(1000, 500):.6f}")

### Q129: Tool-use Safety (Read-only Guard)

In [ ]:
def safe_db_exec(query: str):
    if any(forbidden in query.upper() for forbidden in ["DROP", "DELETE", "UPDATE"]):
        return "ERROR: Write operation blocked."
    return f"SUCCESS: Selected data from query: {query}"

print(safe_db_exec("DELETE FROM users"))
print(safe_db_exec("SELECT * FROM users"))